In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import torch.utils.data as utils
from mpl_toolkits.mplot3d import Axes3D
from PIL import Image
import tensorflow as tf

In [2]:
# Import Dataset
qcd_raw = pd.read_csv('../HiggsReconstruction/EventPlotting/qcd_outputDataForLearning.csv')
hh_raw = pd.read_csv('../HiggsReconstruction/EventPlotting/dihiggs_outputDataForLearning.csv')

print(len(qcd_raw), "rows of qcd data")
print(len(hh_raw), "rows of dihiggs data")
# qcd_raw.head()

# hh_raw.head()

1703 rows of qcd data
4605 rows of dihiggs data


In [3]:
print(hh_raw[0:0]) # get names of all the data
# exclude isMatchable, met, met_phi, scalarHT, nBTags

Empty DataFrame
Columns: [hh_mass, h1_mass, h2_mass, hh_pt, h1_pt, h2_pt, deltaR(h1, h2), deltaR(h1 jets), deltaR(h2 jets), deltaPhi(h1, h2), deltaPhi(h1 jets), deltaPhi(h2 jets), met, met_phi, scalarHT, nJets, nBTags, isMatchable, jet1_pt, jet2_pt, jet3_pt, jet4_pt, jet1_eta, jet2_eta, jet3_eta, jet4_eta, jet1_phi, jet2_phi, jet3_phi, jet4_phi, jet1_mass, jet2_mass, jet3_mass, jet4_mass, jet1_px, jet2_px, jet3_px, jet4_px, jet1_py, jet2_py, jet3_py, jet4_py, jet1_pz, jet2_pz, jet3_pz, jet4_pz, jet1_energy, jet2_energy, jet3_energy, jet4_energy, jet1_btag, jet2_btag, jet3_btag, jet4_btag]
Index: []

[0 rows x 54 columns]


In [4]:
# drop unwanted rows from dataframe
hh_raw = hh_raw.drop(columns=['isMatchable', 
                              'met', 'met_phi', 
                              'nJets', 
                              'nBTags']);
qcd_raw = qcd_raw.drop(columns=['isMatchable', 
                                'met', 'met_phi', 
                                'nJets', 
                                'nBTags']);

In [5]:
hh_all = np.array(hh_raw); qcd = np.array(qcd_raw); n_factors = np.shape(hh_all)[1]; print(n_factors)
image_dim = (7,7)

49


In [6]:
# add labels to di-higgs
hh_all=hh_all[:,:n_factors]
hh_labels= np.zeros((len(hh_all),1))
hh_labels = hh_labels+1
hh_all = np.append(hh_all, hh_labels, axis=1)
# hh_all = np.append(hh_all, 1-hh_labels, axis=1)## hh qcd labels 

qcd=qcd[:,:n_factors]
qcd_labels= np.zeros((len(qcd),1))
qcd = np.append(qcd, qcd_labels, axis=1) 
# qcd = np.append(qcd, 1-qcd_labels, axis=1)# qcd qcd labels

hh = hh_all[0:len(qcd)]

all_data = np.append(hh,qcd, axis=0) 

np.random.seed(0)

all_labels = all_data[:,n_factors:]

all_data = all_data[:,:n_factors]

# scale the data by dividing it by the max value of each
for i in range(np.shape(all_data)[1]):
    all_data[:,i] = np.true_divide(all_data[:,i], np.max(all_data[:,i]))
# print(all_data[:4])

# at this point, there is a dataset with 49 columns of data and 2 columns of labels

In [7]:
all_data.shape

(3406, 49)

In [8]:
# all_data.shape
# all_data_as_pixels = np.ndarray(all_data.shape[0], all_data.shape[1], 3)
value = np.empty((), dtype=object)
value[()] = (0, 0, 0)
all_data_as_pixels = np.zeros((all_data.shape[0], all_data.shape[1], 1), dtype=np.uint8)
for i in range(np.shape(all_data)[1]): 
    for j in range(len(all_data[i])):
#         print(all_data_as_pixels[i][j].shape)
        feature = all_data[i][j]
        all_data_as_pixels[i][j][0] = 255*feature
#         all_data_as_pixels[i][j][1] = 255*feature
#         all_data_as_pixels[i][j][2] = 255*feature

In [9]:
# print(all_data_as_pixels[:4])

In [10]:
# first_img_pixels = all_data_as_pixels[0]
print(all_data_as_pixels[0].shape)
print(all_data_as_pixels.shape)
# first_img_pixels = first_img_pixels.reshape(image_dim[0],image_dim[1],3)
# print(first_img_pixels)

(49, 1)
(3406, 49, 1)


In [19]:
images = []
images_and_labels = []

# images = np.reshape(all_data_as_pixels.shape[0], (image_dim[0],image_dim[1],3))


for pixels, labels in zip(all_data_as_pixels, all_labels):
    ''' zipping ensures that images array and labels array are the same length '''
#     print(pixels.shape)
    img = pixels.reshape(image_dim[0],image_dim[1],1)
#     print(img.shape)
    images.append(img)
# #   images_and_labels.append(np.concatenate((img, labels), axis=1))
print(images[0].shape)
# w, h = 512, 512
# data = np.zeros((h, w, 3), dtype=np.uint8)
# data[0:256, 0:256] = [255, 0, 0] # red patch in upper left
# for i in all_data_as_pixels:
#     print(all_data_as_pixels[i].shape)
#     img_pixels = all_data_as_pixels[i]
#     img_pixels = img_pixels.reshape(image_dim[0],image_dim[1],3)
#     img = Image.fromarray(img_pixels, 'RGB')
#     images[i] = img

# img = Image.fromarray(first_img_pixels, 'RGB')
# img.save('my.png')
# img.show()
# img_0 = Image.fromarray(images[0], 'RGB')
# img_0
img_dataset = [Image.fromarray(img) for img in images]
img_dataset[0]
len(img_dataset)
# images.shape
print(len(img_dataset))
# img_dataset[0]

(7, 7, 1)


TypeError: Cannot handle this data type

In [20]:
# plt.imshow(all_data_as_pixels[0].reshape(image_dim[0],image_dim[1],3), cmap='Greys')
# plt.imshow(images[0], cmap='Greys')
import cv2
print(all_labels[0])

[1.]


In [21]:
images = np.array(images)
x_train, x_test, y_train, y_test = train_test_split(images, all_labels, train_size=0.7, test_size=0.3, random_state=42)
# train_dataset = tf.data.D÷ataset.from_tensor_slices((np.array(images), all_labels))
# test_dataset = tf.data.Dataset.from_tensor_slices((test_examples, test_labels))
train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_data, test_labels))

NameError: name 'train_data' is not defined

In [22]:
# print(len(train_data))
# train_data[0]
images.shape

(3406, 7, 7, 1)

In [16]:
# Reshaping the array to 4-dims so that it can work with the Keras API
img_rows = 7
img_cols = 7
print(x_train.shape)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)
# Making sure that the values are float so that we can get decimal points after division
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# Normalizing the RGB codes by dividing it to the max RGB value.
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print('Number of images in x_train', x_train.shape[0])
print('Number of images in x_test', x_test.shape[0])

(2384, 7, 7, 1)
x_train shape: (2384, 7, 7, 1)
Number of images in x_train 2384
Number of images in x_test 1022


In [17]:
# Importing the required Keras modules containing model and layers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
# Creating a Sequential Model and adding the layers
model = Sequential()
model.add(Conv2D(7, kernel_size=(2,2), input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten()) # Flattening the 2D arrays for fully connected layers
model.add(Dense(50, activation=tf.nn.relu))
model.add(Dropout(0.2))
model.add(Dense(10,activation=tf.nn.softmax))
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
print(x_train.shape)
model.fit(x=x_train,y=y_train, epochs=10)
model.evaluate(x_test, y_test)

Using TensorFlow backend.
W0622 19:59:02.514168 4548347328 deprecation_wrapper.py:119] From /Users/annacuddeback/LHCResearch/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0622 19:59:02.530984 4548347328 deprecation_wrapper.py:119] From /Users/annacuddeback/LHCResearch/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0622 19:59:02.534003 4548347328 deprecation_wrapper.py:119] From /Users/annacuddeback/LHCResearch/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0622 19:59:02.550528 4548347328 deprecation_wrapper.py:119] From /Users/annacuddeback/LHCResearch/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max

(2384, 7, 7, 1)
Epoch 1/10
2384/2384 [==============================] - 0s 188us/step - loss: 1.8212 - acc: 0.5076
Epoch 2/10
2384/2384 [==============================] - 0s 71us/step - loss: 0.7672 - acc: 0.5055
Epoch 3/10
2384/2384 [==============================] - 0s 74us/step - loss: 0.7246 - acc: 0.4870
Epoch 4/10
2384/2384 [==============================] - 0s 57us/step - loss: 0.7018 - acc: 0.5214
Epoch 5/10
2384/2384 [==============================] - 0s 53us/step - loss: 0.6973 - acc: 0.5227
Epoch 6/10
2384/2384 [==============================] - 0s 52us/step - loss: 0.7025 - acc: 0.5034
Epoch 7/10
2384/2384 [==============================] - 0s 50us/step - loss: 0.6977 - acc: 0.5076
Epoch 8/10
2384/2384 [==============================] - 0s 50us/step - loss: 0.6994 - acc: 0.4929
Epoch 9/10
2384/2384 [==============================] - 0s 55us/step - loss: 0.6972 - acc: 0.4904
Epoch 10/10
1022/1022 [==============================] - 0s 81us/step


[0.6832342839520972, 0.495107631744004]

In [ ]:
def get_average_images(img_array, labels):
    width, height, n_channels = img_array[0].shape
    print(width, height, n_channels)
    num_images = len(img_array)
    print(num_images)
    avg_hh_image = np.zeros((width, height, n_channels), np.float)
    avg_qcd_image = np.zeros((width, height, n_channels), np.float)
    n_hh = 0
    n_qcd = 0
    for img, label in zip(img_array, labels): 
        if(label[0]==1):
#             print("HH")
            avg_hh_image = avg_hh_image + img
            n_hh = n_hh +1
        if(label[0]==0):
            avg_qcd_image = avg_qcd_image + img
            n_qcd = n_qcd+1
    avg_hh_image = avg_hh_image/n_hh
    avg_qcd_image = avg_qcd_image/n_qcd
#     print(avg_image)
    avg_hh_image = np.array(np.round(avg_hh_image), dtype=np.uint8)
    avg_qcd_image = np.array(np.round(avg_qcd_image), dtype=np.uint8)
#     print(avg_image)
#     plt.imshow(avg_hh_image, cmap='Greys');
#     plt.imshow(avg_qcd_image, cmap='Greys');
#     print(avg_hh_image)
#     print("____")
#     print(avg_qcd_image)
    return avg_hh_image, avg_qcd_image
hh_avg_img, qcd_avg_img = get_average_images(images, all_labels)

In [ ]:
plt.imshow(hh_avg_img, cmap='Greys')

In [ ]:
plt.imshow(qcd_avg_img, cmap='Greys')